# Proyecto B: Calculadora de ReTweets
### Luis Ángel de Ávila Bula - Juan Ándres Guevara

En este trabajo, se buscará analizar distintos tópicos de un conjunto de tweets usando herramientas de procesamiento de lenguaje natural, tales como Word2Vec y factorización de matrices SVD, entre otras.

Adicionalmente se entrenará un modelo de predicción que permitira dado un tweet, estimar cuantos retweets este tweet tendría.

Para esta tarea, se construye un dataset con la mayor cantidad de tweets en Colombia durante la última semana.

Para ello, seleccionamos un total de 100604 tweets de la API de Twitter publicados del 15 al 21 de Febrero.

Estos tweets se pueden ver plasmados en la carpeta de `data/periodX.csv` donde `X` corresponde al periodo, de 0 a 6, donde cada período es un día en la semana seleccionada.

El proceso de extracción de estos tweets, se encuentran documentados más a fondo en el notebook adjunto a este, de `RecoleccionDatos.ipynb`

Una vez con los datos extraídos, nos preparamos para procesarlos, luego importamos todas las librerías necesarias:

In [1]:
# Libraries
from gensim.models.word2vec import Word2Vec 
import pandas as pd
import numpy as np
import gradio as gr
import re
import nltk
from unidecode import unidecode
from nltk import stem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse.linalg import svds as SparseSVD
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump, load

Procedemos a leer nuestros datos almacenados en el formato csv:

In [6]:
# Dataset
df = pd.concat([pd.read_csv(f'data/period{i}.csv') for i in range(7)])
df = df[["id","lang","tweet", "retweet_count"]]
df.head()

,id,lang,tweet,retweet_count
0,1493736882106245120,es,@JuanCar99077589 @elojodiestro @intiasprilla Y...,0
1,1493736869108006912,es,@NairoQuinCo no tiene ni puta idea como es la ...,0
2,1493736868755611649,es,@Enrique_GomezM deje me decirle con todo res...,0
3,1493736858999787521,es,@PizarroMariaJo @MovimientoMAIS @UP_Colombia @...,0
4,1493736858769108992,es,Mal día para el duquecito en el parlamento eur...,0


Leemos un archivo adicional, que son stop words en español, estas fueron tomadas de https://github.com/Alir3z4/stop-words que contiene una lista de stop words común en distintos lenguajes.

In [7]:
file = open('data/spanish_stop_words.txt',encoding='UTF-8')
SPANISH_STOP_WORDS = file.read().split()
file.close()

Por último antes, de seguir al análisis, definiremos nuestra rutina principal de preprocesamiento de datos, definida como se ve a continuación:

In [8]:
def preprocessing(d):
    # d -> document

    removed = re.sub(r'http\S+', '', d) # removes links from tweet
    words = removed.split()
    # Convert to lowercase every word  
    prep = [w.lower() for w in words]

    # Remove Spanish Stop Words
    prep = [w for w in prep if w not in SPANISH_STOP_WORDS]
    if len(prep) == 0:
        return ''
    # Perform Unidecode normalization for removing accentuations, special characters etc.
    prep = [unidecode(w) for w in prep]

    # Remove special characters and punctuation
    prep = [re.sub('[^A-Za-z0-9]+', '', w) for w in prep]
    
    # Remove empty strings
    prep = [w for w in prep if w != '']
    
    # Lemmatization
    wnl = stem.WordNetLemmatizer()        
    prep = [wnl.lemmatize(w) for w in prep]
    
    
    processed_string = ' '.join(prep)
    return processed_string

Esta es bastante explicativa por si mismo, sin embargo cabe destacar algunos pasos únicos no tan usuales. Se decidio quitar por completo de los tweets todos los links con el patrón de http, y adicionalmente se hizo uso de unidecode para normalizar las palabras.

Dado se esta trabajando en español, no sería deseado perder información valiosa debido a acentuaciones o similares, luego por ejemplo la palabra Bogotá, es preprocesada a bogota, sin acentuación.

Adicionalmente definimos una rutina un poco peculiar, dado nuestro uso de stop words en español, es posible que un tweet contenga solo stop words, es decir, no nos de mucha información.
Aunque es un caso bastante único que ocurre muy pocas veces, es bueno tenerlo presente, por eso definimos una información que nos elimine este tipo de tweets de nuestro dataset, como se ve a continuación:

In [9]:
def clean_noninformant_tweets(dataset):
    # removes those tweets that will yield empty preprocessing, this only happens when tweet is fully contained by stop words
    # Veeeeeeery few tweets actually do this, but it's an important check to have inplace
    indexes = []
    for i in range(len(dataset)):
        if preprocessing(dataset.iloc[i]["tweet"]) == '':
            indexes.append(i)
    dataset.drop(indexes, inplace = True)

## Análisis de Tópicos

Primero realizaremos procesamiento sobre nuestros datos:

In [10]:
tweets = df.tweet
tweets = [x.split() for x in tweets]
corpus = [' '.join(tweets[i]) for i in range(len(tweets))]
corpus[3:5]

['@PizarroMariaJo @MovimientoMAIS @UP_Colombia @PoloDemocratico @petrogustavo @LevyRincon @AquinoTicias1 @Betocoralg @ColombiaHumana_ @somospactometa @PactoCol Cuantos delincuentes en un sola imagen',
 'Mal día para el duquecito en el parlamento europeo. Le tocó quedarse calladito, creyó que ir de paseo por el mundo a nadie le iba importar lo que pasa en Colombia.']

In [11]:
corpus = [preprocessing(d) for d in corpus]
corpus[3:5]

['pizarromariajo movimientomais upcolombia polodemocratico petrogustavo levyrincon aquinoticias1 betocoralg colombiahumana somospactometa pactocol cuantos delincuentes imagen',
 'mal dia duquecito parlamento europeo toco quedarse calladito creyo paseo mundo iba importar pasa colombia']

Una vez con nuestros datos preprocesados, procedemos a hacer uso del vectorizador de TF-IDF, en el que calculamos el TF-IDF por cada par documento / vocabulario.

In [12]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()
print(vocab[10000:100005])
X_tfidf

['amputado' 'amputados' 'amputo' ... 'stroke' 'strong' 'stronger']


<100604x111905 sparse matrix of type '<class 'numpy.float64'>'
	with 1257986 stored elements in Compressed Sparse Row format>

De esta manera para obtener los tópicos solo hace falta realizar la descomposición SVD de esta matriz, y sacar los tópicos de las matrices:

In [13]:
U, s, Vh = SparseSVD(X_tfidf)
Vh = np.array(Vh)
print(U.shape, s.shape, Vh.shape)

(100604, 6) (6,) (6, 111905)


In [14]:
num_top_words = 10

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [15]:
n_topics = 5 # Number of topics
show_topics(Vh[:n_topics])

['colombia presidente pais venezuela petro duque mundo dia seleccion gobierno',
 'petrogustavo colombiahumana upcolombia movimientomais pizarromariajo pactocol polodemocratico somospactometa aquinoticias1 betocoralg',
 'colombia publicar foto acaba petrogustavo bogota petro presidente venezuela pactocol',
 'aborto 24 semana corte legal constitucional despenaliza semanas gestacion despenalizacion',
 'colombia aborto 24 legal semana corte constitucional despenaliza pais acaba']

La factorización de valores singulares es un método para encontrar las principales dimensiones de un conjunto de datos, las dimensiones en la cual se presenta la mayor variación de los datos. La matriz $V$ nos brindó información muy interesante de nuestro conjunto de datos (una matriz tfidf). Al seleccionar las  6 filas de la matriz (tópicos) y al seleccionar las palabras más relevantes de cada una de ellas, hemos obtenido 'clusters' de palabras, que categorizan cada uno de los tweets.

Con las matrices $U$, $S$ y $V$ hemos podido descomponer nuestra matriz término-documento en una matriz cuadrada documento-término ($U$), una matriz diagonal ($S$) con las importancias (varianzas) de cada término y una matriz rectangular término-documento ($V$) de donde pudimos extraer los tópicos.

Los tópicos coinciden con los temas de actualidad que más relevancia están teniendo en el país. Como lo es, las próximas elecciones presidenciales y de congreso, la actividad las redes de Gustavo Petro, los discursos del presidente Duque, la legalización del aborto, etc. Esto nos demuestra que el análisis de tópicos nos puede brindar un muy buen contexto, acerca de lo que tratan cada uno de los documentos de una manera muy sencilla y sin la necesidad de conocer el contenido de cada uno de ellos.

## Predicción de Retweets

Ahora nuestro objetivo es poder predecir el número de retweets según un día dado.
Es decir, dado un tweet cuantos retweets se podría esperar que este tuviera. Asimismo, como realizar un análisis sobre cuales palabras, son las que están produciendo más retweets, según el día y claramente la tendencia.

Para esto construimos un Pipeline a seguir por cada día (conjunto de tweets).
Se hará uso de la técnica de Word2Vec, para transformar todos los tweets de un día, a un espacio embebido de vectores, y luego ajustar un regresor de Random Forest a los datos, para así dado un nuevo vector (tweet) predecir cuantos retweets se podría esperar que este va a tener.

Solamente existen dos detalles controversiales a esta técnica, los cuales veremos a continuación:

Primero, Word2Vec realiza embebimientos sobre palabras, no sobre oraciones, como tweets.
Para esto debemos determinar una manera de asignarle a un tweet a un solo vector que sea representativo, para esto se decide usar el promedio de los vectores correspondientes a cada palabra del tweet, como se ve a continuación:



In [16]:
def vectorized_tweet(model, tweet):
    vectorized_tweet = []
    for word in tweet:
        vectorized_tweet.append(model.wv.get_vector(word))
    return np.average(vectorized_tweet,axis=0)

Segundo, Word2Vec es un embebimiento para un conjunto dado de palabras. Si buscamos realizar una predicción sobre cualquier otro tweet que contenga una palabra desconocida al modelo, no podremos realizar la predicción. 

Afortunadamente, la librería Gensim permite actualizar el modelo de Word2Vec con nuevos vocabularios y reentrenarlos de manera eficiente. Por lo cual, cuando se presente una nueva palabra al modelo, solamente hace falta actualizar el modelo existente para incluirla.

Esto lo podemos hacer a través de las siguientes funciones que verifican que las palabras existan en el modelo, y si no es así, lo actualiza acordemente.
Una función es para un solo tweet, y otra es para multiples, por términos de simplicidad y eficiencia al actualizar el modelo.


In [17]:
def getNewTweetVector(model, tweet): # Intented for a single tweet 
  tweet = preprocessing(tweet)
  words = tweet.split(' ')
  missing_words = [x for x in words if x not in model.wv.key_to_index]
  if len(missing_words) > 0:
    ## adding a word to the model
    model.build_vocab([missing_words], update=True)
    model.train([missing_words], total_examples=model.corpus_count, epochs=model.epochs)
  return vectorized_tweet(model, words)

def getNewTweetVectorsMultiple(model, tweets): # Intended for a multiple list of tweets, for more efficiency
    list_of_tweets = [preprocessing(tw) for tw in tweets]
    list_of_words = [tw.split(' ') for tw in list_of_tweets]
    new_sentences = [] # only add sentences which aren't already in the model.
    for sentence in list_of_words:
        for word in sentence:
          if word not in model.wv.key_to_index:
            new_sentences.append(sentence)
            break 
    
    if len(new_sentences) > 0:
        print(f"Adding {len(new_sentences)} new sentences to model")
        ## add new sentences to model
        model.build_vocab(new_sentences, update=True)
        model.train(new_sentences, total_examples=model.corpus_count, epochs=model.epochs)
    # get vectorized tweets with our newly updated model
    return np.row_stack([vectorized_tweet(model, x) for x in list_of_words])

Con esos detalles solucionados, solo hace falta diseñar nuestro pipeline de procesamiento, como fue descrito anteriormente.
Dado un corpus (conjunto de tweets), y un y, el número de retweets de cada tweet correspondiente.

Se realiza preprocesamiento de los tweets, crea un modelo word2vec, y posteriormente se entrena con un regresor de Random Forest, como lo vemos a continuación:


In [18]:
def BuildAndTrainModel(corpus, y, vector_size=500, min_count = 1, workers = 20, sg=1, window = 30, sample = 1e-6, random_forest_depth = 3, random_seed = 42):
    '''Receives Corpus and y
    Corpus is a list of documents. In this context a list of tweets
    Y is the regression target for the given document. In this context, the number of retweets.
    INCLUDES PREPROCESSING
    Builds a word2vec model with given parameters, and then creates and trains Random Forest Regressor

    Returns: The trained word2vec model, and the fitted random forest regressor'''
    prep = [preprocessing(d) for d in corpus]
    
    sentences = []
    for document in prep:
        sentences.append(document.split())
    model = Word2Vec(sentences=sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg, sample=sample,
            seed=random_seed)

    X = np.row_stack([vectorized_tweet(model, x) for x in sentences])
    regr = RandomForestRegressor(max_depth = random_forest_depth, random_state = random_seed, n_jobs=-1)
    regr.fit(X,y)
    return model, regr

Por último haremos pruebas sobre estos modelos, y es necesario determinar que tan buenos son, para esto usaremos la medida de RMSE (Root Mean Squared Error), la cual calcularemos con la siguiente función:

In [19]:
def TestModelRMSE(model, regr, test_tweets, y_test):
    X_test = getNewTweetVectorsMultiple(model, test_tweets)
    y_predicted = regr.predict(X_test)
    return mean_squared_error(y_test, y_predicted, squared=False)

Ahora pongamos a prueba nuestros modelos diseñados, todos los hiperpárametros fueron seleccionados *empiricamente*. 

Por cada conjunto de tweets de cada día, desde febrero 15 hasta febrero 21 (inclusivo), se realiza una separación de conjunto de datos de prueba, y datos de entrenamiento.

Un 80% de los tweets son tomados para entrenar y el 20% restante para probar que tan bien se desarrolla el modelo ante nuevos datos.
Cabe destacar, que esta separación se realiza previamente a cualquier procesamiento sobre los datos, para asegurar que sea totalmente imparcial ante los datos nuevos. 

Es decir, una vez entrenado el modelo, para predecir sobre los datos de prueba, es necesario realizar preprocesamiento sobre ellos, añadirlos al modelo de Word2Vec, actualizar el modelo, y luego realizar la predicción con el regresor de Random Forest. Justo como se realizaría con un tweet totalmente nuevo.

De esta manera, se realiza el pipeline y se calcula el RMSE como medida de error, y luego se calcula el promedio total de estos, a través de los 7 días.

Cabe destacar que este proceso puede ser un poco demorado, dependiendo del procesador de la maquina puede tomar entre 2-10m, los resultados estarán plasmados en una tabla inferior.

Esto lo podemos ver a continuación:

In [20]:
avg_rmse = 0
for i in range(7):
  day_df = pd.read_csv(f'data/period{i}.csv')
  clean_noninformant_tweets(day_df)
  X_train, X_test, y_train, y_test = train_test_split(day_df["tweet"], day_df["retweet_count"], test_size=0.2, random_state=42)
  model, regr = BuildAndTrainModel(X_train,y_train, vector_size=350, min_count = 1, workers = 1, sg=1, window = 10, 
                                  sample = 1e-3, random_forest_depth = 2, random_seed=42)
  rmse = TestModelRMSE(model, regr, X_test, y_test)
  avg_rmse += rmse
  print(f"Tweets from Feb {15 + i} - Feb {15 + i + 1}(2022) RMSE = ", TestModelRMSE(model, regr, X_test, y_test))
avg_rmse /= 7
print(f"Total Average RMSE across all 7 days: {avg_rmse}:")

Adding 1832 new sentences to model
Tweets from Feb 15 - Feb 16(2022) RMSE =  109.92081235828661
Adding 1798 new sentences to model
Tweets from Feb 16 - Feb 17(2022) RMSE =  68.24988670391909
Adding 1847 new sentences to model
Tweets from Feb 17 - Feb 18(2022) RMSE =  49.422624497886424
Adding 1830 new sentences to model
Tweets from Feb 18 - Feb 19(2022) RMSE =  38.48191579663727
Adding 1844 new sentences to model
Tweets from Feb 19 - Feb 20(2022) RMSE =  51.37659100685054
Adding 1812 new sentences to model
Tweets from Feb 20 - Feb 21(2022) RMSE =  30.704698178423232
Adding 1469 new sentences to model
Tweets from Feb 21 - Feb 22(2022) RMSE =  64.14250785694945
Total Average RMSE across all 7 days: 58.89986234270753:


| Day | RMSE |
| --- | --- | 
| Feb 15 | 109.9208 |
| Feb 16 | 68.2498 | 
| Feb 17 | 49.4226 | 
| Feb 18 | 38.4819 | 
| Feb 19 | 51.3765 | 
| Feb 20 | 30.7659 | 
| Feb 21 | 64.14250 | 
| Average | 58.8998 | 

Cabe destacar que representa el error promedio, dado en las unidades originales. Es decir, se puede interpretar en unidades como retweets.

Así, en promedio, nuestras predicciones tienen un error de alrededor de 59 retweets. Lo cual nos da una idea *decente* de cómo funciona nuestro modelo.

Twitter es muy variado, y los retweets tienden a ser muy dispersos, con muchos retweets teniendo casi que 0 a 5,10 retweets y no más, y con muchos datos atípicos que algunos tweets explotan y tienen 15 mil o 20 mil retweets.

¡Al considerar nuestro error para un tweet que alcanza miles de retweets, el error se podría considerar muy bueno!
Si obtengo más de 2mil retweets un error de 50 no afecta mucho.

Mientras que, en el otro lado, se podría considerar que un si se predice que un tweet va tener 5 retweets, el error de 58 retweets es gigantesco.

Esto se debe a que nuestro modelo es basado en palabras, frecuencias y oraciones, no va más allá y no considera detalles como el orden, usuarios conexiones entre muchos otros factores complejos que podrían determinar realmente la cantidad de retweets.

Por ejemplo, a modo de ilustración, sí un nuevo usuario de la plataforma pública exactamente el mismo tweet que el presidente, es (bastante) probable que el tweet del presidente reciba mayor retweets, que el del nuevo usuario. Este tipo de ocurrencias confunde a nuestro modelo, ya que no es capaz de determinar si el tweet públicado se debería asociar a una alta o baja cantidad de retweets.

Pero no todo es tan malo! Nos puede ayudar a analizar ciertas tendencias del día como lo veremos a continuación:

### Palabras que Maximizan los Retweets

Un beneficio de que nuestro modelo este construido en base de palabras es que es fácil predecir por cada palabra su salida y así determinar aquellas palabras que más se asocian al retweeteo.

Esto se podría interpretar como aquellos tweets que estén más cerca a estas palabras en el espacio embebido, son más probables de obtener retweets.

Podría significar palabras controversiales del día o sencillamente que fueron retweeatadas muchas veces, pero en algunas ocasiones nos dan un *insight* de las tendencias del día. 

Para esto, tomamos nuestro modelo y regresor entrenado, y por cada palabra del modelo le obtenemos su predicción, luego las organizamos de mayor a menor para ver las de mayor valor de retweeteo.

En este caso, para mejorar nuestros modelos lo entrenaremos una vez más, pero esta vez con todo el conjunto de datos de cada día.
Adicionalmente, la mayoría se encuentra pre-entrenado y almacenado en los datos, para que este proceso no tome mucho tiempo:
(Dado el límite de 500mb, se tuvieron que recortar dos modelos ya entrenados)


In [21]:
##### Train and Store
# for i in range(7):
#   day_df = pd.read_csv(f'data/period{i}.csv')
#   clean_noninformant_tweets(day_df)
#   model, regr = BuildAndTrainModel(day_df["tweet"], day_df["retweet_count"], vector_size=350, min_count = 1, workers = 1, sg=1, window = 10, 
#                                   sample = 1e-3, random_forest_depth = 2, random_seed=42)
#   dump((model, regr), f'data/trained_model_{i}')

Cargamos los datos:

In [2]:
model0, regr0 = load('data/trained_model_0')
model1, regr1 = load('data/trained_model_1')
model2, regr2 = load('data/trained_model_2')
model3, regr3 = load('data/trained_model_3')
model4, regr4 = load('data/trained_model_4')
model5, regr5 = load('data/trained_model_5')
model6, regr6 = load('data/trained_model_6')
models = [model0, model1, model2, model3, model4, model5, model6]
regressors = [regr0, regr1, regr2 ,regr3, regr4 , regr5, regr6]

Definimos una función que nos ayudara a mostrar los datos *side_by_side*:

In [3]:
from IPython.core.display import HTML
from IPython.display import display
### Taken and adapted from https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        try:
            output += df.style.set_table_attributes("style='display:inline'").set_caption(caption).hide()._repr_html_()
        except:
            output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

Realizamos la predicción para todas las palabras de cada modelo, y tomamos las top 10:

In [4]:
top10df = []
for model, regr in zip(models, regressors):
  words2retweet = pd.DataFrame(columns = ["words", "retweet_value"])
  words2retweet["words"] = model.wv.index_to_key
  words2retweet["retweet_value"] = regr.predict(model.wv.vectors)
  df = words2retweet.sort_values(by=['retweet_value'], ascending=False)[:20]
  top10df.append(df)

Las mostramos de forma amigable:

In [5]:
display_side_by_side(top10df, [f'Feb {15 + i}' for i in range(7)])

,words,retweet_value
135,parlamento,3829.248177
2551,apreciaciones,3728.793554
150,europarles,3500.677164
1808,reclama,3494.457406
2816,erroneas,3490.845204
191,europeo,3356.019245
194,lideres,3277.079741
761,europea,3190.184071
279,eurodiputados,3118.198776
17,ivanduque,3105.976882


Lo cual es bastante impresionante a pesar de ser un método de ser basado exclusivamente de conteo de palabras.

Por ejemplo concentremosnos en el 21 febrero, día en el que se dio a conocer la controversial noticia de la despenalización del aborto en Colombia.

Podemos ver que las palabras con mayor retweets son muy relacionadas a la tendencia del día, como embarazo, aborto, feto, malformación, interrupción, voluntaria entre otras.

Un análisis similar se podría realizar para los otros días según sus tendencias, pero el ejemplo del 21 de febrero se puede evidenciar claramente.

## Calculadora de Retweets
Ahora podemos ver la calculadora en acción con una interfaz gráfica amigable:

In [6]:
def calculator(DayOfWeek, Tweet):
    if DayOfWeek == "Feb15":
        return float(regr0.predict(getNewTweetVector(model0, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb16":
        return float(regr1.predict(getNewTweetVector(model1, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb17":
        return float(regr2.predict(getNewTweetVector(model2, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb18":
        return float(regr3.predict(getNewTweetVector(model3, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb19":
        return float(regr4.predict(getNewTweetVector(model4, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb20":
        return float(regr5.predict(getNewTweetVector(model5, Tweet).reshape(1,-1))[0])
    elif DayOfWeek == "Feb21":
        return float(regr6.predict(getNewTweetVector(model6, Tweet).reshape(1,-1))[0])

iface = gr.Interface(
    calculator,
    [gr.inputs.Radio(["Feb15", "Feb16", "Feb17", "Feb18", "Feb19", "Feb20", "Feb21"]), "text"],
    "number",
    title="ReTweet Calculator",
    description="Introduzca un tweet y seleccione un día, la calculadora predecirá los retweets del tweet ingresado si se hubiese publicado en un día de la semana.",
    allow_flagging="never"
)

iface.launch()

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x21f9bd41eb0>,
 'http://127.0.0.1:7862/',
 None)

## Conclusiones

De este trabajo podemos concluir:

- El procesamiento de lenguaje natural tiene muchas aplicaciones, y este es un ejemplo de cómo lo podemos aplicar a analizar tendencias, tópicos de cosas del día a día.

- Podemos ver como al mezclar conceptos tan básicos como conteos y frecuencias, es posible realizar un análisis de tópicos de un conjunto de datos, con ayuda de herramientas como la descomposición SVD. Sin realmente entrar a detallar estos tópicos y sin ir a leer 100 mil tweets, es posible resumir las tendencias en unos tópicos.

- Las aplicaciones de Word2Vec son increíbles y fascinantes, el concepto de embeber algo tan natural como palabras, a un espacio tan matemático de $\mathbb{R}^{n}$ es algo que funciona muy bien y tiene demasiadas aplicaciones.

- Las puertas que nos abre Word2Vec son todas las aplicaciones de aprendizaje automático de máquina, como lo vimos en este caso, con un regresor de Random Forest. Con este se pudo alcanzar a realizar predicciones decentes y ayudarnos a analizar aún más que palabras producen retweets, asimismo predecir cuantos retweets tendrá un tweet antes de publicarlo, acercándonos más a las tendencias.

- Los resultados obtenidos son acorde a la realidad, en esta última semana ha habido bastante movida sobre el aborto y bastante movimiento político dada las elecciones de senado que se acercan en Colombia.


In [26]:
regr0.predict(getNewTweetVector(model0, "sample tweet").reshape(1,-1))[0]

4.211074589074458